## Задача 2. Численное решение. Статистическое моделирование

In [1]:
import numpy as np
import plotly.graph_objects as go

experiments_num = 10000000

Для решения задачи (численного или внвлитического) очень важно определить - что такое случайная хорда.

### Первое определение случайной хорды
За случайное расположение хорд определим, что хорда является реализацией случайного вектора $\xi = (\xi_1, \xi_2)$, где $\xi_i$~Uniform(0,$2\pi$).

Явная формулировка задачи позволяет воспользоваться методом статистического моделирования. Используя случайные величины сгенерированные из Uniform(0,$2\pi$), воспользуемся ими для получения декартовых координат концов хорд, на единичном круге, с R=1 (по соображениям симметрии, решения для фиксированного радиуса будет решением в общем случае).

In [2]:
Chord_1 = np.array([np.random.uniform(0, 2*np.pi), np.random.uniform(0, 2*np.pi)])
Chord_2 = np.array([np.random.uniform(0, 2*np.pi), np.random.uniform(0, 2*np.pi)])
print('Полученные случайные величины:', *Chord_1, *Chord_1)

Полученные случайные величины: 0.131020725742054 2.059731649983871 0.131020725742054 2.059731649983871


Используем формулы:
$x=cos(pi)$,
$y=sin(pi)$.

In [3]:
Chord_1_xy = np.array([(np.cos(Chord_1[0]), np.sin(Chord_1[0])), (np.cos(Chord_1[1]), np.sin(Chord_1[1]))])
Chord_2_xy = np.array([(np.cos(Chord_2[0]), np.sin(Chord_2[0])), (np.cos(Chord_2[1]), np.sin(Chord_2[1]))])

In [4]:
print(f'Chord 1: radian = {Chord_1}, a = {Chord_1_xy[0]}, b = {Chord_1_xy[1]}')
print(f'Chord 2: radian = {Chord_2}, a = {Chord_2_xy[0]}, b = {Chord_2_xy[1]}')

Chord 1: radian = [0.13102073 2.05973165], a = [0.99142906 0.13064619], b = [-0.46968622  0.88283342]
Chord 2: radian = [1.35783932 1.37241271], a = [0.21135103 0.97741022], b = [0.19708491 0.98038642]


Мы получили две случайные хорды, заданные координатами в декартовом базисе. Изобразим их на графике вместе с заданными окружностями.

In [21]:
x1 = np.linspace(Chord_1_xy[0,0], Chord_1_xy[1,0], 100)
y1 = np.linspace(Chord_1_xy[0,1], Chord_1_xy[1,1], 100)

x2 = np.linspace(Chord_2_xy[0,0], Chord_2_xy[1,0], 100)
y2 = np.linspace(Chord_2_xy[0,1], Chord_2_xy[1,1], 100)

fig = go.Figure()

fig.update_xaxes(range=[-1.2, 1.2], zeroline=False)
fig.update_yaxes(range=[-1.2, 1.2])

fig.add_trace(go.Scatter(x=x1, y=y1, line_color="DarkBlue", name = '1-ая хорда'))

fig.add_trace(go.Scatter(x=x2, y=y2, line_color="DarkGreen", name = '2-ая хорда'))

fig.update_layout(
    shapes=[
        dict(
            type="circle",
            xref="x",yref="y",
            x0=-1, y0=-1,
            x1= 1, y1= 1,
            line_color="LightSeaGreen",
        ),
        dict(
            type="circle",
            xref="x",yref="y",
            x0=-0.5, y0=-0.5,
            x1= 0.5, y1= 0.5,
            line_color="DarkRed",
        ),
    ]
)

fig.update_layout(width=800, height=800)

fig.show()

В единичном эксперименте хорды могут не пересекаться ни внутри внешней, ни внутри малой окружности. Проведя эксперимент достаточно большое число раз, мы (следуя <b>закону больших чисел</b>) можем получить вероятность, близкую к истинной при достаточно большом количестве экспериментов.

Функция <b>check_inner</b> проверяет принадлежность точки, заданной декартовыми координатами, области, ограниченной окружностью. 

In [6]:
def check_inner(x,y):
    if (x*x+y*y<=1/4):
        return 1
    else:
        return 0

Функция <b>line_intersect</b> находит точку пересечения двух прямых, описанных уравнением $ax + by + c = 0$, где:<br>
$$a = (y_2-y_1),$$
$$b = (x_1-x_2),$$
$$c = x_1(y_1-y_2) + y_1(x_2-x_1),$$
где $(x_1, y_1), (x_2,y_2)$ - координаты концов отрезка, принадлежащих этой прямой.

Для нахождения точки пересечения решим систему уравнений:
\begin{cases}
a_1x + b_1y + c_1 = 0; \\
a_2x + b_2y + c_2 = 0;
\end{cases}

Точка пересечения существует, если определитель $D = \begin{vmatrix} a_1 & b_1 \\ a_2 & b_2 \end{vmatrix}\neq0$.

В таком случае, используя формулы Крамера, получаем, что $x = \frac{D_x}{D}$, $y = \frac{D_y}{D}$ - точка пересечения прямых, где $D_x = \begin{vmatrix} -с_1 & b_1 \\ -c_2 & b_2 \end{vmatrix}$, $D_y = \begin{vmatrix} a_1 & -c_1 \\ a_2 & -c_2 \end{vmatrix}$

In [7]:
def line_intersect(Ax1, Ay1, Ax2, Ay2, Bx1, By1, Bx2, By2):
    a1 = Ay2-Ay1
    b1 = Ax2-Ax1
    c1 = Ax1*(Ay1-Ay2)+Ay1*(Ax2-Ax1)
    
    a2 = By2-By1
    b2 = Bx2-Bx1
    c2 = Bx1*(By1-By2)+By1*(Bx2-Bx1)  
    
    D = a1*b2-b1*a2
    
    if D:
        Dx = -c1*b2 +c2*b1
        Dy = a1*(-c2) +a2*c1
        return Dx/D, Dy/D
    else:
        return

Точка пересечения и истинность ее принадлежности внутреннему кругу для рисунка выше:

In [22]:
pt = line_intersect(*Chord_1_xy[0],*Chord_1_xy[1],*Chord_2_xy[0],*Chord_2_xy[1])
if (pt!=None):
    print('Точка пересечения прямых: ({:0.4f},{:0.4f})'.format(pt[0], pt[1]), 
          check_inner(*pt)*'Хорды пересекаются во внутреннем круге')

Точка пересечения прямых: (0.0036,-1.5217) 


Используя описанные выше функции <b>line_intersect</b> и <b>check_inner</b>, проведем $10^8$ экспериментов, параллельно аккамулируя случаи попадания точки пересечения прямых во внутренний круг. 

Важный момент, точка пересечения прямых может быть как в пределах внешнего круга, так и вне его, то есть точка пересечения прямых - не обязательно точка пересечения хорд, на которых эти прямые построены, что может вызывать вопрос. Однако, в случае пересечения прямых любом из кругов, это обязательно будет точкой пересечения хорд.

In [9]:
%%time
inner_intersection = 0
for i in range(experiments_num):
    Chord_1 = np.array([np.random.uniform(0, 2*np.pi), np.random.uniform(0, 2*np.pi)])
    Chord_2 = np.array([np.random.uniform(0, 2*np.pi), np.random.uniform(0, 2*np.pi)])
    
    Chord_1_xy = np.array([(np.cos(Chord_1[0]), np.sin(Chord_1[0])), (np.cos(Chord_1[1]), np.sin(Chord_1[1]))])
    Chord_2_xy = np.array([(np.cos(Chord_2[0]), np.sin(Chord_2[0])), (np.cos(Chord_2[1]), np.sin(Chord_2[1]))])

    pt = line_intersect(*Chord_1_xy[0],*Chord_1_xy[1],*Chord_2_xy[0],*Chord_2_xy[1])
    if (pt!=None):
        inner_intersection +=check_inner(*pt)

Wall time: 5min 7s


Результат моделирования для $10^8$ экспериментов:

In [10]:
print('Вероятность пересечения во внутреннем круге: '+str(inner_intersection/experiments_num))

Вероятность пересечения во внутреннем круге: 0.0542846


Таким образом была получена доля успешных экспериментов, и следуя идее метода статистического моделирования это можно приблизительным ответом: 
$$P(Хорды~пересекаются~во~внутреннем~круге) \approx 0.054$$
Однако, прежде чем делать выводы рассмотрим другие определения случайного задания хорд.

### Второе определение случайной хорды

Теперь за случайное расположение хорд будем рассматривать хорду проходящую через пару случайных точек, являющихся реализацией радиус-вектора $\xi = (\xi_1, \xi_2)$, где $\xi_1$ ~ Uniform (0,$2\pi$) определяет поворот этого вектора, $\xi_2$ ~ Uniform (0,$1$) определяет его длину.

In [11]:
Chord_1 = np.array([np.random.uniform(0, 2*np.pi), np.random.uniform(0, 2*np.pi), 
                        np.random.uniform(0, 1), np.random.uniform(0, 1)])
Chord_2 = np.array([np.random.uniform(0, 2*np.pi), np.random.uniform(0, 2*np.pi), 
                        np.random.uniform(0, 1), np.random.uniform(0, 1)])
    
Chord_1_xy = np.array([(np.cos(Chord_1[0])*Chord_1[2], np.sin(Chord_1[0])*Chord_1[2]),
                           (np.cos(Chord_1[1])*Chord_1[3], np.sin(Chord_1[1])*Chord_1[3])])
Chord_2_xy = np.array([(np.cos(Chord_2[0])*Chord_2[2], np.sin(Chord_2[0])*Chord_2[2]),
                           (np.cos(Chord_2[1])*Chord_1[3], np.sin(Chord_2[1])*Chord_1[3])])

x1 = np.linspace(Chord_1_xy[0,0], Chord_1_xy[1,0], 100)
y1 = np.linspace(Chord_1_xy[0,1], Chord_1_xy[1,1], 100)

x2 = np.linspace(Chord_2_xy[0,0], Chord_2_xy[1,0], 100)
y2 = np.linspace(Chord_2_xy[0,1], Chord_2_xy[1,1], 100)

fig = go.Figure()

fig.update_xaxes(range=[-1.2, 1.2], zeroline=False)
fig.update_yaxes(range=[-1.2, 1.2])

fig.add_trace(go.Scatter(x=x1, y=y1, line_color="DarkBlue", name = '1-ая хорда'))

fig.add_trace(go.Scatter(x=x2, y=y2, line_color="DarkGreen", name = '2-ая хорда'))

fig.update_layout(
    shapes=[
        dict(
            type="circle",
            xref="x",yref="y",
            x0=-1, y0=-1,
            x1= 1, y1= 1,
            line_color="LightSeaGreen",
        ),
        dict(
            type="circle",
            xref="x",yref="y",
            x0=-0.5, y0=-0.5,
            x1= 0.5, y1= 0.5,
            line_color="DarkRed",
        ),
    ]
)

fig.update_layout(width=800, height=800)


fig.show()

In [12]:
%%time
intersection = 0
inner_intersection = 0
for i in range(experiments_num):
    Chord_1 = np.array([np.random.uniform(0, 2*np.pi), np.random.uniform(0, 2*np.pi), 
                        np.random.uniform(0, 1), np.random.uniform(0, 1)])
    Chord_2 = np.array([np.random.uniform(0, 2*np.pi), np.random.uniform(0, 2*np.pi), 
                        np.random.uniform(0, 1), np.random.uniform(0, 1)])
    
    Chord_1_xy = np.array([(np.cos(Chord_1[0])*Chord_1[2], np.sin(Chord_1[0])*Chord_1[2]),
                           (np.cos(Chord_1[1])*Chord_1[3], np.sin(Chord_1[1])*Chord_1[3])])
    Chord_2_xy = np.array([(np.cos(Chord_2[0])*Chord_2[2], np.sin(Chord_2[0])*Chord_2[2]),
                           (np.cos(Chord_2[1])*Chord_1[3], np.sin(Chord_2[1])*Chord_1[3])])
        
    pt2 = line_intersect(*Chord_1_xy[0],*Chord_1_xy[1],*Chord_2_xy[0],*Chord_2_xy[1])
    if (pt2!=None):
        inner_intersection +=check_inner(*pt2)


Wall time: 6min 58s


In [13]:
print('Вероятность пересечения во внутреннем круге: '+str(inner_intersection/experiments_num))

Вероятность пересечения во внутреннем круге: 0.5597173


### Третье определение случайной хорды

Рассмотрим хорду, проходящую через две точки, каждая из которых является реализацией случайного вектора: $\xi = (\xi_1, \xi_2)$, где $\xi_1$ ~ Uniform$(-1,1)$ - координата по оси абсцисс, $\xi_2$~Uniform$(-\sqrt[2]{1-\xi_1^2},\sqrt[2]{1-\xi_1^2})$ - координата по оси ординат.

In [14]:
%%time
intersection = 0
inner_intersection = 0
for i in range(experiments_num):
    
    x1, x2 = np.random.uniform(-1, 1), np.random.uniform(-1, 1)
    y1 =  np.random.uniform(-np.sqrt(1-x1*x1), np.sqrt(1-x1*x1))
    y2 = np.random.uniform(-np.sqrt(1-x2*x2), np.sqrt(1-x2*x2))
    Chord_1_xy = np.array([(x1, y1),(x2, y2)])

    x1, x2 = np.random.uniform(-1, 1), np.random.uniform(-1, 1)
    y1 =  np.random.uniform(-np.sqrt(1-x1*x1), np.sqrt(1-x1*x1))
    y2 = np.random.uniform(-np.sqrt(1-x2*x2), np.sqrt(1-x2*x2))
    Chord_2_xy = np.array([(x1, y1),(x2, y2)])
        
    pt = line_intersect(*Chord_1_xy[0],*Chord_1_xy[1],*Chord_2_xy[0],*Chord_2_xy[1])
    if (pt!=None):
        inner_intersection +=check_inner(*pt)


Wall time: 6min 6s


In [15]:
print('Вероятность пересечения во внутреннем круге: '+str(inner_intersection/experiments_num))

Вероятность пересечения во внутреннем круге: 0.2554311


### Четвертое определение случайной хорды

Аналогично первому определению будем рассматривать хорду как реализацию случайной пары точек на окружности в радианах, однако вторая точка отталкивается от реализации первой: $\xi = (\xi_1, \xi_1+\xi_2)$, где $\xi_i$~Uniform(0,$2\pi$). Определение случайности вызывает мысли об аналогии результата с первым случаем. Проверим смоделировав данные эксперименты.

In [16]:
f, s = np.random.uniform(0, 2*np.pi), np.random.uniform(0, 2*np.pi) 
Chord_1 = np.array([f, f + np.random.uniform(0, 2*np.pi)])
Chord_2 = np.array([s, s + np.random.uniform(0, 2*np.pi)])
Chord_1_xy = np.array([(np.cos(Chord_1[0]), np.sin(Chord_1[0])), (np.cos(Chord_1[1]), np.sin(Chord_1[1]))])
Chord_2_xy = np.array([(np.cos(Chord_2[0]), np.sin(Chord_2[0])), (np.cos(Chord_2[1]), np.sin(Chord_2[1]))])


x1 = np.linspace(Chord_1_xy[0,0], Chord_1_xy[1,0], 100)
y1 = np.linspace(Chord_1_xy[0,1], Chord_1_xy[1,1], 100)

x2 = np.linspace(Chord_2_xy[0,0], Chord_2_xy[1,0], 100)
y2 = np.linspace(Chord_2_xy[0,1], Chord_2_xy[1,1], 100)

fig = go.Figure()

fig.update_xaxes(range=[-1.2, 1.2], zeroline=False)
fig.update_yaxes(range=[-1.2, 1.2])

fig.add_trace(go.Scatter(x=x1, y=y1, line_color="DarkBlue", name = '1-ая хорда'))

fig.add_trace(go.Scatter(x=x2, y=y2, line_color="DarkGreen", name = '2-ая хорда'))

fig.update_layout(
    shapes=[
        dict(
            type="circle",
            xref="x",yref="y",
            x0=-1, y0=-1,
            x1= 1, y1= 1,
            line_color="LightSeaGreen",
        ),
        dict(
            type="circle",
            xref="x",yref="y",
            x0=-0.5, y0=-0.5,
            x1= 0.5, y1= 0.5,
            line_color="DarkRed",
        ),
    ]
)

fig.update_layout(width=800, height=800)

fig.show()

In [17]:
%%time

def check_outer(x,y):
    if (x*x+y*y<=1):
        return 1
    else:
        return 0
    
inner_intersection = 0
outer_intersection = 0
inner_both = 0
inner_1 = 0
inner_2 = 0


for i in range(experiments_num):
    f, s = np.random.uniform(0, 2*np.pi), np.random.uniform(0, 2*np.pi) 
    Chord_1 = np.array([f, f + np.random.uniform(0, 2*np.pi)])
    Chord_2 = np.array([s, s + np.random.uniform(0, 2*np.pi)])
    Chord_1_xy = np.array([(np.cos(Chord_1[0]), np.sin(Chord_1[0])), (np.cos(Chord_1[1]), np.sin(Chord_1[1]))])
    Chord_2_xy = np.array([(np.cos(Chord_2[0]), np.sin(Chord_2[0])), (np.cos(Chord_2[1]), np.sin(Chord_2[1]))])
    
    b1 = check_inner(*Chord_1_xy.mean(axis=0))
    b2 = check_inner(*Chord_2_xy.mean(axis=0))
    if(b1&b2):
        inner_both +=1
    if b1:
        inner_1 +=1
    if b2:
        inner_2 +=1
        
    pt = line_intersect(*Chord_1_xy[0],*Chord_1_xy[1],*Chord_2_xy[0],*Chord_2_xy[1])
    if (pt!=None):
        outer_intersection +=check_outer(*pt)
        inner_intersection +=check_inner(*pt)


Wall time: 8min 59s


In [18]:
print('Доля пересечения хорд во внутреннем круге:  {:0.4f}'.format(inner_intersection/experiments_num))

Доля пересечения хорд во внутреннем круге:  0.0543


In [19]:
print('Доля пересечения хорд во внешнем круге:  {:0.4f}'.format(outer_intersection/experiments_num))

Доля пересечения хорд во внешнем круге:  0.3334


In [20]:
print('Доля ситуаций с попаданием обеих хорд во внутреннюю окружность: {:0.4f}'.format(inner_both/experiments_num))

Доля ситуаций с попаданием обеих хорд во внутреннюю окружность: 0.1112


## Выводы из статистического решения
Четвертый и первый способ задания хорд выдает приблизительно одну долю пересечений во внутреннем круге (0.054), заметно отличающеюся от той, что получается вторым (0.559) и третьим (0.255) способом. Задача является примером <b>парадокса Бертрана</b>: на геометричекую вероятность влияет то, как именно задается случайность построения хорды в задаче. 

## Размышление на аналитическое решение

В четвертом случае также были вычислены экспериментальные доли того, как часто в случае соответвующего способа задания случайности задания хорды <b>1) две случайные хорды пересекаются во внешнем круге</b>, <b>2) две случайные хорды попадают во внутренний круг</b>.

Аналитически можно вывестивероятность того, что случайная хорда попадет во внутреннюю окружность. Из предположения симметрии зафиксируем один из концов хорды на окружности. Тогда второй конец хорды может быть равномерно распределен на 2$\pi$ от первого. 

Тогда вероятность того, что хорда пересечет внутреннюю окружность равна вероятности того, что вторая точка окажется на дуге окружности ограниченной двукмя касательными проведенными из первой точки к внутренней окружности. Изобразить вывод того, что угол между касательными составляет 60&deg; здесь сложно, но это выводится несложными геометрическими выкладками. Тогда искомая дуга относится ко всей окружности как $\frac{1}{3}$, и это будет вероятностью случайно заданой хорде пересечь внутренню окружность.

Тогда вероятность того, что обе хорды пересекут внутренний круг, в связи с тем что пересечение его по отдельности есть независимые события, равно произведению вероятностей по отдельности. То есть $\frac{1}{9}$ (что и было получено числено выше).

Пусть событие А - обе хорды пересеклись внутри меньшего круга, событие В - обе хорды попали во внутренний круг, для второго полученная вероятность $\frac{1}{9}$. Рассмотрим условные вероятности:
$$P(A|B) = \frac{P(A \cap B)}{P(B)} = \frac{P(B \cap A)}{\frac{1}{9}}  = 9 * P(B|A)P(A)$$
Вероятность того, что хорды попадут во внутренний круг при условии того, что они пересклись в нем - P(B|A) - равна 1. Тогда вероятность искомого события A:
$$P(A) = \frac{P(A|B)}{9}$$
И остается необходимым найти вероятность того, что <b>хорды пересекутся внутри маленькой окружности, при условии, что обе попали в нее</b>....